In [1]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('spark_basics').getOrCreate()

22/02/09 10:23:02 WARN Utils: Your hostname, ganesh-pi resolves to a loopback address: 127.0.1.1; using 192.168.1.119 instead (on interface eth0)
22/02/09 10:23:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/09 10:23:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.json('people.json')

In [5]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [6]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [7]:
df.columns

['age', 'name']

In [9]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [11]:
#Assign the schema when default schema is wrong
from pyspark.sql.types import (StructField, StringType, 
                                IntegerType, StructType)

data_schema = [StructField('age', IntegerType(), True), 
                StructField('name', StringType(), True)]

In [12]:
#Assigning the created schema to a final structure
final_struc = StructType(fields=data_schema)

In [13]:
df = spark.read.json('people.json', schema=final_struc)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



GRABBING DATA FROM DFS

In [14]:
df['age']

Column<'age'>

In [15]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [17]:
df.select('age').head(2)[0]

Row(age=None)

In [19]:
df.withColumn('new_age', df['age']*2).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     60|
|  19| Justin|     38|
+----+-------+-------+



In [20]:
#To just rename a column use withColumnRenamed 
df.withColumnRenamed('age', 'new_age').show()

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



In [21]:
#USE SQL with pyspark
df.createOrReplaceTempView('people')

In [22]:
results = spark.sql("SELECT * FROM people")

In [24]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [25]:
new_results = spark.sql("SELECT * FROM people WHERE age=30")
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



BASIC OPERATIONS WITH SPARK DATAFRAMES 

In [26]:
spark = SparkSession.builder.appName('ops').getOrCreate()

In [29]:
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True)
df.show(truncate=False)

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|Date      |Open              |High              |Low               |Close             |Volume   |Adj Close         |
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998        |214.499996        |212.38000099999996|214.009998        |123432400|27.727039         |
|2010-01-05|214.599998        |215.589994        |213.249994        |214.379993        |150476200|27.774976000000002|
|2010-01-06|214.379993        |215.23            |210.750004        |210.969995        |138040000|27.333178000000004|
|2010-01-07|211.75            |212.000006        |209.050005        |210.58            |119282800|27.28265          |
|2010-01-08|210.299994        |212.000006        |209.06000500000002|211.98000499999998|111902700|27.464034         |
|2010-01-11|212.79999700000002|213.000002        |208.45

In [30]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [34]:
df.filter("Close < 500").select(['Open', 'Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [35]:
df.filter(df['Close'] < 500).select('Volume').show()

+---------+
|   Volume|
+---------+
|123432400|
|150476200|
|138040000|
|119282800|
|111902700|
|115557400|
|148614900|
|151473000|
|108223500|
|148516900|
|182501900|
|153038200|
|152038600|
|220441900|
|266424900|
|466777500|
|430642100|
|293375600|
|311488100|
|187469100|
+---------+
only showing top 20 rows



In [39]:
#Multi case filtering criteria
df.filter( (df.Close < 200) & (df.Open > 200) ).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [43]:
result = df.filter(df.Low == 197.16).collect()
result

[Row(Date='2010-01-22', Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

In [47]:
row = result[0]
row = row.asDict()
row

{'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

GROUPBY AND AGGREGATE FUNCTIONS IN PYSPARK

In [2]:
spark = SparkSession.builder.appName('ags').getOrCreate()

22/02/10 10:06:15 WARN Utils: Your hostname, ganesh-pi resolves to a loopback address: 127.0.1.1; using 192.168.1.119 instead (on interface eth0)
22/02/10 10:06:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/10 10:06:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [4]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [7]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [8]:
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [10]:
df.agg({'Sales':'sum', 'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [11]:
#Utilising the groupby and agg method together
group_data = df.groupBy('Company')
group_data.agg({'Sales':'min'}).show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [18]:
#To use multiple functions on a dataframe do this
from pyspark.sql import functions as f
from pyspark.sql.functions import countDistinct, avg, stddev, format_number
group_data.agg(f.min('Sales'), f.max('Sales')).show()

+-------+----------+----------+
|Company|min(Sales)|max(Sales)|
+-------+----------+----------+
|   APPL|     130.0|     750.0|
|   GOOG|     120.0|     340.0|
|     FB|     350.0|     870.0|
|   MSFT|     124.0|     600.0|
+-------+----------+----------+



In [17]:
#Another way of having aggregate functions in your display
df.select(f.countDistinct('Sales').alias('Unique'), f.avg('Sales').alias('Avergae Sales')).show()

+------+-----------------+
|Unique|    Avergae Sales|
+------+-----------------+
|    11|360.5833333333333|
+------+-----------------+



In [21]:
#Aliasing a column names and formatting the decimal place
df.select(f.format_number(f.stddev('Sales'),2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



In [22]:
#Ordering values
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [23]:
#Order by descending
df.orderBy(df.Sales.desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



HANDLING MISSING DATA

In [24]:
spark = SparkSession.builder.appName('missing').getOrCreate()
df = spark.read.csv('ContainsNull.csv', header=True, inferSchema=True)
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [26]:
#drop missing data with a row having atleast 2 non-null values
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [27]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [28]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [29]:
#Drop rows with nulls in that particular column
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [30]:
#Filling in missing values into the dataframe
df.na.fill('FILL VALUE').show()     #This will fill in missing values only into string columns

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL VALUE| null|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [32]:
df.na.fill(0, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [38]:
#Filling in the missing values with the mean value of the column
df.na.fill(df.select(f.mean('Sales')).collect()[0][0], subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



DATES AND TIMESTAMPS

In [41]:
spark = SparkSession.builder.appName('dates').getOrCreate()

df = spark.read.csv('appl_stock.csv', header=True, inferSchema=True)
df.select(['Date', 'Open']).show()

+----------+------------------+
|      Date|              Open|
+----------+------------------+
|2010-01-04|        213.429998|
|2010-01-05|        214.599998|
|2010-01-06|        214.379993|
|2010-01-07|            211.75|
|2010-01-08|        210.299994|
|2010-01-11|212.79999700000002|
|2010-01-12|209.18999499999998|
|2010-01-13|        207.870005|
|2010-01-14|210.11000299999998|
|2010-01-15|210.92999500000002|
|2010-01-19|        208.330002|
|2010-01-20|        214.910006|
|2010-01-21|        212.079994|
|2010-01-22|206.78000600000001|
|2010-01-25|202.51000200000001|
|2010-01-26|205.95000100000001|
|2010-01-27|        206.849995|
|2010-01-28|        204.930004|
|2010-01-29|        201.079996|
|2010-02-01|192.36999699999998|
+----------+------------------+
only showing top 20 rows



In [40]:
df.head(1)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [43]:
from pyspark.sql.functions import (dayofmonth, hour, dayofyear, month, year,
                                    weekofyear, format_number, date_format)

df.select(f.month(df.Date)).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+
only showing top 20 rows



In [48]:
#Adding in the year column to the dataframe
new_df = df.withColumn("Year", f.year(df.Date))
new_df.groupBy('Year').mean()\
        .select(['Year', f.format_number('avg(Close)', 2).alias('Avg Closing price')])\
        .show()

+----+-----------------+
|Year|Avg Closing price|
+----+-----------------+
|2015|           120.04|
|2013|           472.63|
|2014|           295.40|
|2012|           576.05|
|2016|           104.60|
|2010|           259.84|
|2011|           364.00|
+----+-----------------+

